<a href="https://colab.research.google.com/github/yeesem/Natural-Laguage-Processing/blob/main/Text_Feature_Extraction%2CVectorization_SMS_Spam_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile 1.txt
This is a story about cats
our feline pets
Cats are furry animals

Overwriting 1.txt


In [14]:
%%writefile 2.txt
This story is about surfing
Catching waves is fun
Surfing is a popular water sport

Overwriting 2.txt


In [15]:
with open('1.txt') as f:
  doc = f.read()

In [16]:
print(doc)

This is a story about cats
our feline pets
Cats are furry animals



In [17]:
#Assign an index to every word
vocab = {}
i = 1

with open('1.txt') as f:
  x = f.read().lower().split()

for word in x:
  if word in vocab:
    continue
  else:
    vocab[word] = i
    i+=1

#Print vocab distionary
print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12}


In [18]:
with open('2.txt') as f:
  x = f.read().lower().split()
  for word in x:
    if word in vocab:
      continue
    else:
      vocab[word] = i
      i+=1

#Print vocab dictionary
print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12, 'surfing': 13, 'catching': 14, 'waves': 15, 'fun': 16, 'popular': 17, 'water': 18, 'sport': 19}


# **Feature Extraction**

In [19]:
#Create an empty vector with space for each word in the vocabulary:
one = ['1.txt'] + [0]*len(vocab)

print(one)

['1.txt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
#Mape the frequencies of each word in 1.txt to our vector:
with open('1.txt') as f:
  x = f.read().lower().split()

  for word in x:
    one[vocab[word]]+=1

print(one)

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]


In [21]:
two = ['2.txt'] + [0]*len(vocab)

with open('2.txt') as f:
  x = f.read().lower().split()

  for word in x:
    two[vocab[word]] +=1

In [22]:
print(one , "\n" , two)

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] 
 ['2.txt', 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1]


# **Working on the SMS Spam Collection Dataset**

In [23]:
import numpy as np
import pandas as pd

In [24]:
df = pd.read_csv("/content/smsspamcollection.tsv",sep = '\t')

In [25]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [26]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [27]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X = df['message']
y = df['label']

In [30]:
XTrain,XTest,YTrain,YTest = train_test_split(X,y,test_size = 0.33,random_state = 42)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
count_vect = CountVectorizer()

In [33]:
#FIT VECTORIZER TO THE DATA

#Vectorizer to sentences into to columns

#First Approach
#count_vect.fit(XTrain)
#XTrain_counts = count_vect.transform(XTrain)

#Alternative Approach
XTrain_counts = count_vect.fit_transform(XTrain)

In [34]:
#3733 number of sentences
#7082 number of unique words
XTrain_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [35]:
XTrain.shape

(3733,)

In [36]:
XTrain_counts.shape

(3733, 7082)

In [37]:
from sklearn.feature_extraction.text import TfidfTransformer

In [38]:
tfidf_transformer = TfidfTransformer()

In [39]:
#Transform the vector by adding the number of counts and appearance of the words
XTrain_tfidf = tfidf_transformer.fit_transform(XTrain_counts)

In [40]:
XTrain_tfidf.shape

(3733, 7082)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
#XTrain direct transform to train_tfidf
vectorizer = TfidfVectorizer()
XTrain_tfidf = vectorizer.fit_transform(XTrain)

In [43]:
XTrain_tfidf

<3733x7082 sparse matrix of type '<class 'numpy.float64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [44]:
from sklearn.svm import LinearSVC

In [45]:
clf = LinearSVC()

In [47]:
clf.fit(XTrain_tfidf,YTrain)

LinearSVC()

In [48]:
from sklearn.pipeline import Pipeline

In [49]:
#Able to perform vectorization and classifition by single line of code
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [50]:
#More convenient compared to previous method
text_clf.fit(XTrain,YTrain)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [51]:
prediction = text_clf.predict(XTest)

In [53]:
from sklearn.metrics import confusion_matrix,classification_report

In [55]:
print(confusion_matrix(YTest,prediction))

[[1586    7]
 [  12  234]]


In [56]:
#Perform better by using the vectorization method
print(classification_report(YTest,prediction))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [57]:
from sklearn import metrics

In [59]:
metrics.accuracy_score(prediction,YTest)

0.989668297988037

In [66]:
text_clf.predict(['Hi! How are you doing today?'])

array(['ham'], dtype=object)

In [69]:
text_clf.predict(["Congratulation! You've been selected as a winner. TEXT WON to 44255 cogratulations."])

array(['spam'], dtype=object)